<a href="https://colab.research.google.com/github/tonykipkemboi/Sentiment_Analysis_using_BERT_Neural_Network/blob/main/Kenyan_Embassy_BERT_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install & Import Dependencies

In [3]:
!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio===0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████                   | 834.1MB 1.5MB/s eta 0:13:11tcmalloc: large alloc 1147494400 bytes == 0x5579e9410000 @  0x7f02634a4615 0x5579b066fcdc 0x5579b074f52a 0x5579b0672afd 0x5579b0763fed 0x5579b06e6988 0x5579b06e14ae 0x5579b06743ea 0x5579b06e67f0 0x5579b06e14ae 0x5579b06743ea 0x5579b06e332a 0x5579b0764e36 0x5579b06e2853 0x5579b0764e36 0x5579b06e2853 0x5579b0764e36 0x5579b06e2853 0x5579b0764e36 0x5579b07e73e1 0x5579b07476a9 0x5579b06b2cc4 0x5579b0673559 0x5579b06e74f8 0x5579b067430a 0x5579b06e23b5 0x5579b06e17ad 0x5579b06743ea 0x5579b06e23b5 0x5579b067430a 0x5579b06e23b5
     |████████████████▌               | 1055.7MB 1.3MB/s eta 0:12:16tcmalloc: large alloc 1434370048 bytes == 0x557a2da66000 @  0x7f02634a4615 0x5579b066fcdc 0x5579b074f52a 0x5579b0672afd 0x5579b0763fed 0x5579b06e6988 0x5579b06e14ae 0x5579b06743ea 0x5579b06e67f0 0x5579b06e14ae 0x5579b06743ea 0x5579b06e332a 0x5579b0764e36 0x5579b06e2853 0

In [4]:
!pip3 install transformers requests beautifulsoup4 pandas numpy

     |████████████████████████████████| 2.5MB 28.1MB/s 
     |████████████████████████████████| 901kB 35.6MB/s 
     |████████████████████████████████| 3.3MB 33.8MB/s 


In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re

# Instantiate Model

In [6]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Encode & Calculate Sentiment

In [7]:
tokens = tokenizer.encode('love', return_tensors='pt')

In [8]:
result = model(tokens)

In [9]:
result.logits

tensor([[-1.4507, -1.5098, -0.3014,  0.8204,  1.9042]],
       grad_fn=<AddmmBackward>)

In [10]:
int(torch.argmax(result.logits))+1

5

# Collect Reviews

In [11]:
r = requests.get('https://www.yelp.com/biz/embassy-of-kenya-washington')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [12]:
reviews[0]

'I expected mediocre service and bad attitudes when I visited there but instead I was surprised by the professionalism of the staff there. They were very helpful and patient when dealing with customers and not at all what we are used to. I was shocked and amazed, and I want to congratulate them on a job well done.'

# Load Reviews into DataFrame & Score

In [13]:
df = pd.DataFrame(np.array(reviews), columns=['review'])
df.tail()

,review
3,Useless Useless Useless. Don't expect any help...
4,The embassy decided to join in the tour so lat...
5,New rule for Kenyans abroad is that you must h...
6,Calls will NEVER be returned if you have a que...
7,"Well, when my aunt visited Kenya last year, sh..."


In [14]:
df['review'].iloc[0]

'I expected mediocre service and bad attitudes when I visited there but instead I was surprised by the professionalism of the staff there. They were very helpful and patient when dealing with customers and not at all what we are used to. I was shocked and amazed, and I want to congratulate them on a job well done.'

In [15]:
def sentiment_score(review):
  tokens = tokenizer.encode(review, return_tensors='pt')
  result = model(tokens)
  return int(torch.argmax(result.logits))+1

In [16]:
sentiment_score(df['review'].iloc[1])

1

In [17]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [18]:
df

,review,sentiment
0,I expected mediocre service and bad attitudes ...,4
1,People were waiting before 10 am. They didn't ...,1
2,Calls and emails never get answered! Applied f...,1
3,Useless Useless Useless. Don't expect any help...,1
4,The embassy decided to join in the tour so lat...,2
5,New rule for Kenyans abroad is that you must h...,1
6,Calls will NEVER be returned if you have a que...,1
7,"Well, when my aunt visited Kenya last year, sh...",4


In [19]:
# Review #1
df['review'].iloc[0]

'I expected mediocre service and bad attitudes when I visited there but instead I was surprised by the professionalism of the staff there. They were very helpful and patient when dealing with customers and not at all what we are used to. I was shocked and amazed, and I want to congratulate them on a job well done.'

In [20]:
# Review #2
df['review'].iloc[1]

"People were waiting before 10 am. They didn't allow us to view till 10:15 and only in their driveway and small side yard.Goods for sale and food. Ten dollars for a plate and I didn't like any of it. We were given a taste of tea and coffee. I got a cup of local lavender tea for a keurig machine. \xa0Disappointing."

In [21]:
# Review #3
df['review'].iloc[2]

"Calls and emails never get answered! Applied for Kenyan passport in January 2020 and this is August, no response yet! Even with Covid 19, it shouldn't take this long!! Some folks applied for ID's in January 2020 at this embassy in person, not yet received them. Were promised to be ready in 6 to 8 weeks, cargo planes are still flying from kenya to usa and vice versa, what's the hold up??"

In [22]:
# Review #4
df['review'].iloc[3]

"Useless Useless Useless. Don't expect any help with anything. I called twice and got no where. I emailed and got no where. What is the point of this embassy???"

In [23]:
# Review #5
df['review'].iloc[4]

"The embassy decided to join in the tour so late that they didn't even make it into the tour brochure. \xa0I wish I had walked to the front of the line to notice that we weren't even going inside the embassy. \xa0We waited 45 minutes to walk to the side of the embassy. \xa0The pictures on display were beautiful. \xa0The embassy had run out of food and drinks. \xa0It was disappointing."

In [24]:
# Review #6
df['review'].iloc[5]

'New rule for Kenyans abroad is that you must have a current Kenyan ID.There is NO rules or directions for the requirement to get this ID that would be ONLY valid in Kenya should you need to own property. I was advised over the phone to fly over 1000miles in order to have my fingerprints taken. \xa0So I did only to get there and since I am married to an American that they needed my American husbands fingerprints WHAT? WHY? so my trip to DC was useless now who is going to compensate my unwanted trip this is information 1. should have been listed on the website 2. could have been conveyed on the phone 3. explain why they need fingerprints of a none Kenyan. IF it is because of consent then they should have asked for an affidavit or his presence with witness of his signature. People get married and change names if they please. If my name is legal in the US, I have proof to be Kenyan by birth AND I have a current passport why would I need proof of who I am after all this how did I get my pa

In [25]:
# Review #7
df['review'].iloc[6]

'Calls will NEVER be returned if you have a questions. There is an answering machine that is no good if you cant get an answer. No live person during WORKING hours. I have to fly ALL the way to Washington DC just to get an answer.'

In [26]:
# Review #8
df['review'].iloc[7]

'Well, when my aunt visited Kenya last year, she had a great time - of course. \xa0The embassy was there to make sure that she left here with little problem, and from what I was told it was a smooth trip.'